In [1]:
using DataFrames
using Chemfiles

home = "/home/german/labo/18/egfr/model/"

"/home/german/labo/18/egfr/model/"

In [2]:
function res_3_to_1(in_3::AbstractString)
    
    res = lowercase(in_3)
    out_1 = AbstractString
    
    if res == "ala"
        out_1 = "A"
    elseif res == "arg"
        out_1 = "R"
    elseif res == "asn"
        out_1 = "N"
    elseif res == "asp"
        out_1 = "D"
    elseif res == "cys"
        out_1 = "C"
    elseif res == "gln"
        out_1 = "Q"
    elseif res == "glu"
        out_1 = "E"
    elseif res == "gly"
        out_1 = "G"
    elseif res == "his"
        out_1 = "H"
    elseif res == "ile"
        out_1 = "I"
    elseif res == "leu"
        out_1 = "L"
    elseif res == "lys"
        out_1 = "K"
    elseif res == "met"
        out_1 = "M"
    elseif res == "phe"
        out_1 = "F"
    elseif res == "pro"
        out_1 = "P"
    elseif res == "ser"
        out_1 = "S"
    elseif res == "thr"
        out_1 = "T"
    elseif res == "trp"
        out_1 = "W"
    elseif res == "tyr"
        out_1 = "Y"
    elseif res == "val"
        out_1 = "V"
    else
        error("Unknown aminoacid: ", in_3)
    end
    
    return out_1
end

function res_3_to_1(in_vec::Array{String, 1})

    out_vec = Array{String, 1}(length(in_vec))
    i = 0
    for each in in_vec
        i+=1
        try 
            out_vec[i] = res_3_to_1(each)
        catch e
            error("Malformed array list.\nCaught error: $e")
        end
    end
    
    return out_vec
end

res_3_to_1 (generic function with 2 methods)

In [3]:
function seq_from_top(in_top::Chemfiles.Topology)
    # Get 1 letter code amino acid list from a topology
    aa = convert(Int64, count_residues(in_top))
    ids = Array{Int64, 1}(aa)

    for i=1:aa
        ids[i] = id(Residue(in_top, i-1))
    end
    ids_sorted_indices = sortperm(ids);

    seq_3 = Array{String, 1}(aa)
    i = 0
    for each in ids_sorted_indices
        i+=1
        seq_3[i] = name((Residue(in_top, each-1)))
    end
    seq_1 = res_3_to_1(seq_3);
    
    return seq_1
end

seq_from_top (generic function with 1 method)

In [4]:
function write_pdb_to_pir(in_top::Chemfiles.Topology, in_pdb_filename::AbstractString,
        out_filename::AbstractString)

    aa = convert(Int64, count_residues(in_top))
    ids = convert(Array{Int64, 1}, [ id(Residue(in_top, i-1)) for i=1:aa ])
    primer_aa = minimum(ids)
    ultimo_aa = maximum(ids)
    seq = seq_from_top(in_top)

    open(out_filename, "w") do f
        # Header
        write(f, string(">P1;", in_pdb_filename, "\n"))
        write(f, string("structureX:", in_pdb_filename, ":", primer_aa, ":A:+", aa, ":A:::-1.00:-1.00\n"))
        # Sequence
        nlineas = convert(Int64, floor(aa / 80))
        for i = 1:nlineas
            uno = (i-1) * 80 + 1
            ult = i * 80
            write(f, seq[uno:ult])
            write(f, "\n")
        end
        uno = nlineas * 80 + 1
        write(f, seq[uno:end])
        write(f, "*\n")
    end
end

write_pdb_to_pir (generic function with 1 method)

# Escribo secuencias modelo

In [5]:
tmp = readdlm(string(home, "/pdbs.list"))
pdbs_list = convert(Array{String, 1}, reshape(tmp, length(tmp)))

for each in pdbs_list
    println(each)
    in_pdb_trj = Trajectory(string(home, each, "/", each, ".pdb"))
    in_pdb_frm = read(in_pdb_trj)
    in_pdb_top = Topology(in_pdb_frm)
    seq = seq_from_top(in_pdb_top)
    salida_filename = string(home, each, "/", each, ".pir")

    write_pdb_to_pir(in_pdb_top, each, salida_filename)
end

1M14_A
1M17_A
1XKK_A
2EB2_A
2GS2_A
2GS6_A
2GS7_A
2ITN_A
2ITP_A
2ITU_A
2ITX_A
2ITZ_A
2J5F_A
2RGP_A
3BEL_A
3GOP_A
3GT8_A
3GT8_C
3IKA_A
3IKA_B
3LZB_A
3UG1_A
3VJN_A
3W2O_A
3W2R_A
3W2S_A
3W32_A
3W33_A
4G5J_A
4I1Z_A
4I22_A
4I23_A
4I24_A
4LI5_A
4LQM_A
4R3P_A
4R5S_A
4RJ4_A
4ZAU_A
4ZJV_A
5C8K_A
5CAO_A
5CAP_A
5CAV_A
5CNN_A
5CNN_B
5CNO_A
5CZH_A
5EDP_A
5EM5_A
5EM6_A
5EM7_A
5EM8_A
5HG5_A
5HG7_A
5HG8_A
5HIB_A
5HIC_A


# Alineo

In [6]:
for each in pdbs_list
    println(each)
    out_filename = string(home, each, "/alignment_", each)
    in_a = string(home, each, "/canonical")
    in_b = string(home, each, "/", each, ".pir")
    run(`needle $in_a $in_b -gapopen 10.0 -gapextend 0.5 $out_filename`)
end

1M14_A
1M17_A
1XKK_A
2EB2_A
2GS2_A


Needleman-Wunsch global alignment of two sequences
Needleman-Wunsch global alignment of two sequences
Needleman-Wunsch global alignment of two sequences
Needleman-Wunsch global alignment of two sequences


2GS6_A
2GS7_A
2ITN_A
2ITP_A
2ITU_A
2ITX_A
2ITZ_A
2J5F_A


Needleman-Wunsch global alignment of two sequences
Needleman-Wunsch global alignment of two sequences
Needleman-Wunsch global alignment of two sequences
Needleman-Wunsch global alignment of two sequences
Needleman-Wunsch global alignment of two sequences
Needleman-Wunsch global alignment of two sequences
Needleman-Wunsch global alignment of two sequences
Needleman-Wunsch global alignment of two sequences


2RGP_A
3BEL_A
3GOP_A
3GT8_A
3GT8_C
3IKA_A
3IKA_B
3LZB_A


...

Needleman-Wunsch global alignment of two sequences
Needleman-Wunsch global alignment of two sequences
Needleman-Wunsch global alignment of two sequences
Needleman-Wunsch global alignment of two sequences
Needleman-Wunsch global alignment of two sequences
Needleman-Wunsch global alignment of two sequences
Needleman-Wunsch global alignment of two sequences
Needleman-Wunsch global alignment of two sequences


5CAV_A
5CNN_A
5CNN_B
5CNO_A
5CZH_A
5EDP_A
5EM5_A


Needleman-Wunsch global alignment of two sequences
Needleman-Wunsch global alignment of two sequences
Needleman-Wunsch global alignment of two sequences
Needleman-Wunsch global alignment of two sequences
Needleman-Wunsch global alignment of two sequences
Needleman-Wunsch global alignment of two sequences
Needleman-Wunsch global alignment of two sequences


5EM6_A
5EM7_A
5EM8_A
5HG5_A
5HG7_A
5HG8_A
5HIB_A
5HIC_A


Needleman-Wunsch global alignment of two sequences
Needleman-Wunsch global alignment of two sequences
Needleman-Wunsch global alignment of two sequences
Needleman-Wunsch global alignment of two sequences
Needleman-Wunsch global alignment of two sequences
Needleman-Wunsch global alignment of two sequences
Needleman-Wunsch global alignment of two sequences
Needleman-Wunsch global alignment of two sequences
Needleman-Wunsch global alignment of two sequences


# Leo alineamiento

In [7]:
alineamientos = Array{Array{Any, 2}, 1}(length(pdbs_list))
indices_mut = Array{Array{Int64, 1}, 1}(length(pdbs_list))
indices_mis = Array{Array{Int64, 1}, 1}(length(pdbs_list))

j = 0

for each in pdbs_list
    j += 1
    println(each) 
    
    in_seq_canonical = Array{AbstractString, 1}
    in_seq_each = Array{AbstractString, 1}
    in_mut_each = Array{AbstractString, 1}

    seq_canonical = Array{AbstractString, 1}
    seq_each = Array{AbstractString, 1}
    mut_each = Array{AbstractString, 1}
    
    
    # Leo archivo.
    in_filename = string(home, each, "/alignment_", each)
    alignment_file = open(in_filename)
    lineas = readlines(alignment_file)

    k = 0
    # Ahora leo todas las líneas y tomo los arrays en bruto
    for linea in lineas
        k+=1
        inicio = 31
        final = length(lineas) - 3
    
        if inicio < k < final
            n_linea = k - 31
            altura = ((n_linea - 1 + 4) % 4)
        
            if altura == 0
                in_seq_canonical = [ in_seq_canonical ; linea[22:end-7] ]
            elseif altura == 1
                in_mut_each = [ in_mut_each ; linea[22:end] ]
            elseif altura == 2
                in_seq_each = [ in_seq_each ; linea[22:end-7] ]
            end
        end
    end
    
    # Paso las secuencias en bruto a arrays bien formados.
    for i = 2:length(in_seq_canonical)
        tmp_canonical = [ in_seq_canonical[i][j] for j = 1:length(in_seq_canonical[i]) ]
        seq_canonical = [ seq_canonical ; tmp_canonical]
    
        tmp_each = [ in_seq_each[i][j] for j = 1:length(in_seq_each[i]) ]
        seq_each = [ seq_each ; tmp_each]

        tmp = [ in_mut_each[i][j] for j = 1:length(in_mut_each[i]) ]
        mut_each = [ mut_each ; tmp]
    end

    # Determino q pdbs tienen mutaciones y en q posiciones.
    tmp_indices_mut = findin(mut_each, ".")
    indices_mut[j] = tmp_indices_mut
    
    # Determino las regiones a modelar
    tmp_indices_mis = findin(mut_each, " ") .- 1
    indices_mis[j] = tmp_indices_mis

    # Guardo todo
    alineamiento = [ seq_canonical seq_each mut_each ]
    alineamientos[j] = alineamiento 
end

1M14_A
1M17_A
1XKK_A
2EB2_A
2GS2_A
2GS6_A
2GS7_A
2ITN_A
2ITP_A
2ITU_A
2ITX_A
2ITZ_A
2J5F_A
2RGP_A
3BEL_A
3GOP_A
3GT8_A
3GT8_C
3IKA_A
3IKA_B
3LZB_A
3UG1_A
3VJN_A
3W2O_A
3W2R_A
3W2S_A
3W32_A
3W33_A
4G5J_A
4I1Z_A
4I22_A
4I23_A
4I24_A
4LI5_A
4LQM_A
4R3P_A
4R5S_A
4RJ4_A
4ZAU_A
4ZJV_A
5C8K_A
5CAO_A
5CAP_A
5CAV_A
5CNN_A
5CNN_B
5CNO_A
5CZH_A
5EDP_A
5EM5_A
5EM6_A
5EM7_A
5EM8_A
5HG5_A
5HG7_A
5HG8_A
5HIB_A
5HIC_A


# Hago las mutaciones en la secuencia original

In [8]:
k = 0
for each in indices_mut
    k += 1
    if length(each) != 0
        for cada in each
            alineamientos[k][cada, 1] = alineamientos[k][cada, 2]
        end
    end
end

# El array de las regiones a modelar enumera aminoácidos, lo paso a rangos

In [9]:
regiones_mis = Array{Array{Array{Int64, 1}, 1}, 1}(length(pdbs_list))

for i = 1:length(pdbs_list)
    tmp_indices = Array{Int64}[]
    primero = indices_mis[i][1]
    for j = 1:length(indices_mis[i]) - 1

        uno = indices_mis[i][j]
        dos = indices_mis[i][j+1]
        # ### 
        # Checkeo q no pase del 277
        if uno > 277
            break
        elseif dos > 277
            push!(tmp_indices, [ primero, uno ])
            break
        end
        # ###
        if (dos - uno) > 1
            # Non-consecutive numbers        
            push!(tmp_indices, [ primero, uno ])            
            primero = dos            
        elseif j == (length(indices_mis[i]) - 1)
            # Last iteration
            push!(tmp_indices, [ primero, uno ])
        else
            # Consecutive numbers
            continue
        end
    end
    regiones_mis[i] = tmp_indices
end

# Escribo el alineamiento p/ modelar

In [93]:
k = 0
  
for each in pdbs_list
    k += 1
    modelo = string(each, "_full")
    out_filename = string(home, each, "/to_model_", each)
    
    println(each)

    in_pdb_trj = Trajectory(string(home, each, "/", each, ".pdb"))
    in_pdb_frm = read(in_pdb_trj)
    in_pdb_top = Topology(in_pdb_frm)
    
    aa = convert(Int64, count_residues(in_pdb_top))
    npos = length(alineamientos[k][:, 1])
    ids = convert(Array{Int64, 1}, [ id(Residue(in_pdb_top, i-1)) for i=1:aa ])
    primer_aa = minimum(ids)
    ultimo_aa = maximum(ids)
    seq = seq_from_top(in_pdb_top)

    open(out_filename, "w") do f
        nlineas = convert(Int64, ceil(npos / 80))
        
        # 1era secuencia:
        # Header
        write(f, string(">P1;", each, "\n"))
        write(f, string("structureX:", each, ":", primer_aa, ":A:+", aa, ":A:::-1.00:-1.00\n"))
        # Sequence
        
        for i = 1:nlineas
            uno = (i-1) * 80 + 2 # uno y ult tienen un desplazamiento de 1 unidad pq
            ult = i * 80 + 1 # los vectores de las secuencias arrancan con un char vacío
            try
                write(f, alineamientos[k][:, 2][uno:ult])
                write(f, "\n")
            catch e
                write(f, alineamientos[k][:, 2][uno:end])
                write(f, "*\n\n")
            end
        end
    
        # 2da secuencia:
        # Header
        write(f, string(">P1;", modelo, "\n"))
        write(f, string("sequence", ":::::::::\n"))
        # Sequence
        for i = 1:nlineas
            uno = (i-1) * 80 + 2 # uno y ult tienen un desplazamiento de 1 unidad pq
            ult = i * 80 + 1 # los vectores de las secuencias arrancan con un char vacío
            try
                write(f, alineamientos[k][:, 1][uno:ult])
                write(f, "\n")
            catch e 
                write(f, alineamientos[k][:, 1][uno:end])
                write(f, "*\n\n")
            end
        end
    end
end

1M14_A
1M17_A
1XKK_A
2EB2_A
2GS2_A
2GS6_A
2GS7_A
2ITN_A
2ITP_A
2ITU_A
2ITX_A
2ITZ_A
2J5F_A
2RGP_A
3BEL_A
3GOP_A
3GT8_A
3GT8_C
3IKA_A
3IKA_B
3LZB_A
3UG1_A
3VJN_A
3W2O_A
3W2R_A
3W2S_A
3W32_A
3W33_A
4G5J_A
4I1Z_A
4I22_A
4I23_A
4I24_A
4LI5_A
4LQM_A
4R3P_A
4R5S_A
4RJ4_A
4ZAU_A
4ZJV_A
5C8K_A
5CAO_A
5CAP_A
5CAV_A
5CNN_A
5CNN_B
5CNO_A
5CZH_A
5EDP_A
5EM5_A
5EM6_A
5EM7_A
5EM8_A
5HG5_A
5HG7_A
5HG8_A
5HIB_A
5HIC_A


# Escribo el script p/ modelar

In [96]:
k = 0
each = "1XKK_A"

for each in pdbs_list
    k += 1
    
    println(each)

    to_model_filename = string(home, each, "/to_model_", each)
    modelo = string(each, "_full")
    script_filename = string(home, each, "/script_mod_", each, ".py")

    open(script_filename, "w") do script_file
        write(script_file, "from modeller import *\n")
        write(script_file, "from modeller.automodel import *    # Load the automodel class\n")
        write(script_file, "\n")
        write(script_file, "log.verbose()\n")
        write(script_file, "env = environ()\n")
        write(script_file, "\n")
        write(script_file, "# directories for input atom files\n")
        write(script_file, "env.io.atom_files_directory = ['.', '../atom_files']\n")
        write(script_file, "\n")
        write(script_file, "class MyModel(automodel):\n")
        write(script_file, "\tdef select_atoms(self):\n")


        j = 0
        n_regiones = length(regiones_mis[k])
        for cada in regiones_mis[k]
            j += 1
            uno = cada[1]
            dos = cada[2]

            if j == 1
                write(script_file, "\t\treturn selection(self.residue_range('$uno', '$dos'),\n")
            elseif j == n_regiones
                write(script_file, "\t\t\tself.residue_range('$uno', '$dos'),)\n")
            else
                write(script_file, "\t\t\tself.residue_range('$uno', '$dos'),\n")
            end
        end

        write(script_file, "\n")
        write(script_file, "\n")
        write(script_file, "a = MyModel(env, alnfile = '$to_model_filename',\n")
        write(script_file, "            knowns = '$each', sequence = '$modelo')\n")
        write(script_file, "a.starting_model= 1\n")
        write(script_file, "a.ending_model  = 1\n")
        write(script_file, "\n")
        write(script_file, "a.make()\n")
    end
end

1M14_A
1M17_A
1XKK_A
2EB2_A
2GS2_A
2GS6_A
2GS7_A
2ITN_A
2ITP_A
2ITU_A
2ITX_A
2ITZ_A
2J5F_A
2RGP_A
3BEL_A
3GOP_A
3GT8_A
3GT8_C
3IKA_A
3IKA_B
3LZB_A
3UG1_A
3VJN_A
3W2O_A
3W2R_A
3W2S_A
3W32_A
3W33_A
4G5J_A
4I1Z_A
4I22_A
4I23_A
4I24_A
4LI5_A
4LQM_A
4R3P_A
4R5S_A
4RJ4_A
4ZAU_A
4ZJV_A
5C8K_A
5CAO_A
5CAP_A
5CAV_A
5CNN_A
5CNN_B
5CNO_A
5CZH_A
5EDP_A
5EM5_A
5EM6_A
5EM7_A
5EM8_A
5HG5_A
5HG7_A
5HG8_A
5HIB_A
5HIC_A


In [12]:
run(`mod9.19 $script_filename`)

Could not find platform independent libraries <prefix>
Could not find platform dependent libraries <exec_prefix>
Consider setting $PYTHONHOME to <prefix>[:<exec_prefix>]
'import site' failed; use -v for traceback
  File "/home/german/labo/18/egfr/model/1XKK_A/model_1XKK_A.py", line 14
    a = MyModel(env, alnfile = '/home/german/labo/18/egfr/model/1XKK_A/to_model_1XKK_A',
    ^
IndentationError: expected an indented block


LoadError: [91mfailed process: Process(`mod9.19 /home/german/labo/18/egfr/model/1XKK_A/model_1XKK_A.py`, ProcessExited(1)) [1][39m